# Building and deploying machine learning solutions with Vertex AI: Challenge Lab

## Scenario



This lab is recommended for students who have enrolled in the [**Building and deploying machine learning solutions with Vertex AI**](). Are you ready for the challenge?

## Learning objectives

* Train a TensorFlow model locally in a hosted [**Vertex Notebook**](https://cloud.google.com/vertex-ai/docs/general/notebooks?hl=sv).
* Containerize your training code with [**Cloud Build**](https://cloud.google.com/build) and push it to [**Google Cloud Artifact Registry**](https://cloud.google.com/artifact-registry) as part of a Vertex custom container training workflow.
* Deploy your trained model to a [**Vertex Online Prediction Endpoint**](https://cloud.google.com/vertex-ai/docs/predictions/getting-predictions) for serving predictions.
* Request an online prediction and see the response.

## Setup

### Define constants

In [ ]:
# Add installed library dependencies to Python PATH variable.
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin

In [ ]:
# Retrieve and set PROJECT_ID and REGION environment variables.
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]
REGION = 'us-central1'

In [ ]:
# Create a globally unique Google Cloud Storage bucket for artifact storage.
GCS_BUCKET = f"gs://{PROJECT_ID}-vertex-challenge-lab"

In [ ]:
!gsutil mb -la $REGION $GCS_BUCKET

In [ ]:
USER = "dougkelly"  # <---CHANGE THIS
PIPELINE_ROOT = "{}/pipeline_root/{}".format(GCS_BUCKET, USER)

PIPELINE_ROOT

### Import libraries

In [ ]:
import os
import shutil
import json
import logging
from typing import NamedTuple

import tensorflow as tf
import tensorflow_text as text
import tensorflow_hub as hub

from official.nlp import optimization  # to create AdamW optimizer

import pandas as pd
import matplotlib.pyplot as plt

from google.cloud import aiplatform as vertexai

import kfp
from google_cloud_pipeline_components import aiplatform as gcc_aip
from kfp.v2 import dsl
from kfp.v2.dsl import (ClassificationMetrics, Input, Metrics, Model, Output,
                        component)

### Initialize Vertex AI Python SDK

In [61]:
vertexai.init(project=PROJECT_ID, location=REGION, staging_bucket=GCS_BUCKET)

### Import dataset

In [ ]:
url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'

dataset = tf.keras.utils.get_file('aclImdb_v1.tar.gz', url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')

train_dir = os.path.join(dataset_dir, 'train')

# remove unused folders to make it easier to load the data
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [ ]:
# !gsutil -m cp -r {dataset_dir} {GCS_BUCKET}/data/acllmdb

In [42]:
AUTOTUNE = tf.data.AUTOTUNE
batch_size = 32
seed = 42

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

class_names = raw_train_ds.class_names
train_ds = raw_train_ds.cache().prefetch(buffer_size=AUTOTUNE)

val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed)

val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

test_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/test',
    batch_size=batch_size)

test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Found 25000 files belonging to 2 classes.


In [43]:
for text_batch, label_batch in train_ds.take(1):
  for i in range(3):
    print(f'Review: {text_batch.numpy()[i]}')
    label = label_batch.numpy()[i]
    print(f'Label : {label} ({class_names[label]})')

Review: b'"Pandemonium" is a horror movie spoof that comes off more stupid than funny. Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing Saddles", "High Anxiety", and "Spaceballs" are some of my favorite comedies that spoof a particular genre. "Pandemonium" is not up there with those films. Most of the scenes in this movie had me sitting there in stunned silence because the movie wasn\'t all that funny. There are a few laughs in the film, but when you watch a comedy, you expect to laugh a lot more than a few times and that\'s all this film has going for it. Geez, "Scream" had more laughs than this film and that was more of a horror film. How bizarre is that?<br /><br />*1/2 (out of four)'
Label : 0 (neg)
Review: b"David Mamet is a very interesting and a very un-equal director. His first movie 'House of Games' was the one I liked best, and it set a series of films with characters whose perspective of life changes as they

2021-10-07 16:41:36.577696: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


## Choose a BERT model to fine-tune

https://www.tensorflow.org/text/tutorials/classify_text_with_bert#choose_a_bert_model_to_fine-tune

In [ ]:
TFHUB_BERT_PREPROCESSOR = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
TFHUB_BERT_ENCODER = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2"

## Build and train a TensorFlow model

In [ ]:
def build_text_classifier():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessor = hub.KerasLayer(TFHUB_BERT_PREPROCESSOR, name='preprocessing')
    encoder_inputs = preprocessor(text_input)
    encoder = hub.KerasLayer(TFHUB_BERT_ENCODER, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    net = outputs["pooled_output"]      # [batch_size, 512].
    net = tf.keras.layers.Dropout(0.1, name="dropout")(net)
    net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
    return tf.keras.Model(text_input, net)

In [ ]:
text_test = ['this is such an amazing movie!']

In [ ]:
classifier_model = build_text_classifier()
bert_raw_result = classifier_model(tf.constant(text_test))
print(bert_raw_result)

In [ ]:
tf.keras.utils.plot_model(classifier_model)

In [ ]:
loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
metrics = tf.metrics.BinaryAccuracy()

In [ ]:
epochs = 5
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

In [ ]:
# optimizer
classifier_model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                         loss=loss,
                         metrics=metrics)

In [ ]:
# print(f'Training model with {TFHUB_BERT_ENCODER}')
history = classifier_model.fit(x=train_ds,
                               validation_data=val_ds,
                               epochs=epochs)

In [ ]:
loss, accuracy = classifier_model.evaluate(test_ds)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

In [ ]:
history_dict = history.history
print(history_dict.keys())

acc = history_dict['binary_accuracy']
val_acc = history_dict['val_binary_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)
fig = plt.figure(figsize=(10, 6))
fig.tight_layout()

plt.subplot(2, 1, 1)
# "bo" is for "blue dot"
plt.plot(epochs, loss, 'r', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
# plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

## Containerize your model code

In [147]:
MODEL_NAME = "bert-sentiment-classifier"

In [208]:
%%writefile {MODEL_NAME}/trainer/model.py
import os
import shutil
import logging

import tensorflow as tf
import tensorflow_text as text
import tensorflow_hub as hub
from official.nlp import optimization  # to create AdamW optimizer

DATA_URL = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
LOCAL_DATA_DIR = './tmp/data'
AUTOTUNE = tf.data.AUTOTUNE
SEED = 42


def download_data(data_dir):
    """Download dataset."""
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    
    dataset = tf.keras.utils.get_file(
      fname='aclImdb_v1.tar.gz',
      origin=DATA_URL,
      untar=True,
      cache_dir=data_dir,
      cache_subdir="")
    
    dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
    
    train_dir = os.path.join(dataset_dir, 'train')
    
    # remove unused folders to make it easier to load the data
    remove_dir = os.path.join(train_dir, 'unsup')
    shutil.rmtree(remove_dir)
    
    return dataset_dir


def load_datasets(dataset_dir, hparams):
    """Load pre-split tf.datasets.
    Args:
      hparams(dict): A dictionary containing model training arguments.
    Returns:
      raw_train_ds(tf.dataset):
      raw_val_ds(tf.dataset):
      raw_test_ds(tf.dataset):      
    """    

    raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
        os.path.join(dataset_dir, 'train'),
        batch_size=hparams['batch-size'],
        validation_split=0.2,
        subset='training',
        seed=SEED)    

    raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
        os.path.join(dataset_dir, 'train'),
        batch_size=hparams['batch-size'],
        validation_split=0.2,
        subset='validation',
        seed=SEED)

    raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory(
        os.path.join(dataset_dir, 'test'),
        batch_size=hparams['batch-size'])
    
    return raw_train_ds, raw_val_ds, raw_test_ds


def build_text_classifier(hparams, optimizer):
    """Train and evaluate TensorFlow BERT sentiment classifier.
    Args:
      hparams(dict): A dictionary containing model training arguments.
    Returns:
      history(tf.keras.callbacks.History): Keras callback that records training event history.
    """
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessor = hub.KerasLayer(hparams['tf-hub-bert-preprocessor'], name='preprocessing')
    encoder_inputs = preprocessor(text_input)
    encoder = hub.KerasLayer(hparams['tf-hub-bert-encoder'], trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    classifier = outputs["pooled_output"]
    classifier = tf.keras.layers.Dropout(hparams['dropout'], name="dropout")(classifier)
    classifier = tf.keras.layers.Dense(1, activation=None, name='classifier')(classifier)
    
    model = tf.keras.Model(text_input, classifier)   
    
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
    metrics = tf.metrics.BinaryAccuracy()    
    
    model.compile(optimizer=optimizer,
                  loss=loss,
                  metrics=metrics)    
    
    return model


def train_evaluate(hparams):
    """Train and evaluate TensorFlow BERT sentiment classifier.
    Args:
      hparams(dict): A dictionary containing model training arguments.
    Returns:
      history(tf.keras.callbacks.History): Keras callback that records training event history.
    """
    dataset_dir = download_data(LOCAL_DATA_DIR)
    train_ds, val_ds, test_ds = load_datasets(dataset_dir, hparams)
    
    train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
    test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)     
    
    epochs = hparams['epochs']
    steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
    n_train_steps = steps_per_epoch * epochs
    n_warmup_steps = int(0.1 * n_train_steps)    
    
    optimizer = optimization.create_optimizer(init_lr=hparams['initial-learning-rate'],
                                              num_train_steps=n_train_steps,
                                              num_warmup_steps=n_warmup_steps,
                                              optimizer_type='adamw')    
    
    mirrored_strategy = tf.distribute.MirroredStrategy()
    with mirrored_strategy.scope():
        model = build_text_classifier(hparams=hparams, optimizer=optimizer)
        logging.info(model.summary())
        
    history = model.fit(x=train_ds,
                        validation_data=val_ds,
                        epochs=epochs)  
    
    logging.info("Test accuracy: %s", model.evaluate(test_ds))

    # Export Keras model in TensorFlow SavedModel format.
    model.save(hparams['model-dir'])
    
    return history

Overwriting bert-sentiment-classifier/trainer/model.py


In [209]:
%%writefile {MODEL_NAME}/trainer/task.py

import os
import argparse

from trainer import model

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    # Vertex custom container training args. These are set by Vertex AI during training but can also be overwritten.
    parser.add_argument('--model-dir', dest='model-dir',
                        default=os.environ['AIP_MODEL_DIR'], type=str, help='GCS URI for saving model artifacts.')
    # parser.add_argument('--data-dir', dest='data-dir',
    #                     default='gs://dougkelly-vertex-demos-vertex-challenge-lab/data/acllmdb', type=str, help='GCS URI for saving dataset.') 
    parser.add_argument('--tf-hub-bert-preprocessor', dest='tf-hub-bert-preprocessor', 
                        default='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3', type=str, help='TF-Hub URL.')
    parser.add_argument('--tf-hub-bert-encoder', dest='tf-hub-bert-encoder', 
                        default='https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/2', type=str, help='TF-Hub URL.')     

    # Model training args.
    parser.add_argument('--initial-learning-rate', dest='initial-learning-rate', default=3e-5, type=float, help='Learning rate for optimizer.')
    parser.add_argument('--epochs', dest='epochs', default=5, type=int, help='Training iterations.')    
    parser.add_argument('--batch-size', dest='batch-size', default=32, type=int, help='Number of examples during each training iteration.')    
    parser.add_argument('--dropout', dest='dropout', default=0.1, type=float, help='Float percentage of DNN nodes [0,1] to drop for regularization.')    

    
    args = parser.parse_args()
    hparams = args.__dict__

    model.train_evaluate(hparams)

Overwriting bert-sentiment-classifier/trainer/task.py


### Write a `Dockerfile` for your custom model container

In [210]:
%%writefile {MODEL_NAME}/Dockerfile
# Specifies base image and tag.
# https://cloud.google.com/vertex-ai/docs/general/deep-learning
# https://cloud.google.com/deep-learning-containers/docs/choosing-container
FROM gcr.io/deeplearning-platform-release/base-cpu

# Sets the container working directory.
WORKDIR /root

# Copies the requirements.txt into the container to reduce network calls.
COPY requirements.txt .
# Installs additional packages.
RUN pip3 install -U -r requirements.txt

# Copies the trainer code to the docker image.
COPY . /trainer

# Sets the container working directory.
WORKDIR /trainer

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]

Overwriting bert-sentiment-classifier/Dockerfile


In [211]:
%%writefile {MODEL_NAME}/requirements.txt
tensorflow==2.5
tensorflow-text==2.5.0
tf-models-official==2.5.0

Overwriting bert-sentiment-classifier/requirements.txt


## Use Cloud Build to build and submit your container to Google Cloud Artifact Registry

### Create Artifact Repository for custom container images

In [72]:
ARTIFACT_REPOSITORY="bert-sentiment-classifier"

In [73]:
# Create an Artifact Repository using the gcloud CLI.
!gcloud artifacts repositories create $ARTIFACT_REPOSITORY \
--repository-format=docker \
--location=$REGION \
--description="Artifact registry for ML custom training images for sentiment classification"

ERROR: (gcloud.artifacts.repositories.create) ALREADY_EXISTS: the repository already exists


### Create `cloudbuild.yaml` instructions

In [186]:
IMAGE_NAME="bert-sentiment-classifier"
IMAGE_TAG="latest"
IMAGE_URI=f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{ARTIFACT_REPOSITORY}/{IMAGE_NAME}:{IMAGE_TAG}"

In [187]:
cloudbuild_yaml = f"""steps:
- name: 'gcr.io/cloud-builders/docker'
  args: [ 'build', '-t', '{IMAGE_URI}', '.' ]
images: 
- '{IMAGE_URI}'"""

with open(f"{MODEL_NAME}/cloudbuild.yaml", "w") as fp:
    fp.write(cloudbuild_yaml)

### Build and submit your container image to your Artifact Repository

In [212]:
!gcloud builds submit --timeout=20m --config {MODEL_NAME}/cloudbuild.yaml {MODEL_NAME}

Creating temporary tarball archive of 7 file(s) totalling 7.3 KiB before compression.
Uploading tarball of [bert-sentiment-classifier] to [gs://dougkelly-vertex-demos_cloudbuild/source/1633670786.325722-abf2ada3f3024d2cae7d48c11ed5c1be.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/dougkelly-vertex-demos/locations/global/builds/dbf22e46-a66c-498f-b505-e21806846eec].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/dbf22e46-a66c-498f-b505-e21806846eec?project=617979904441].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "dbf22e46-a66c-498f-b505-e21806846eec"

FETCHSOURCE
Fetching storage object: gs://dougkelly-vertex-demos_cloudbuild/source/1633670786.325722-abf2ada3f3024d2cae7d48c11ed5c1be.tgz#1633670786695816
Copying gs://dougkelly-vertex-demos_cloudbuild/source/1633670786.325722-abf2ada3f3024d2cae7d48c11ed5c1be.tgz#1633670786695816...
/ [1 files][  2.8 KiB/  2.8 KiB]                               

In [221]:
import datetime

TIMESTAMP=datetime.datetime.now().strftime('%Y%m%d%H%M%S')
DISPLAY_NAME = "bert-sentiment-{}".format(TIMESTAMP)
GCS_BASE_OUTPUT_DIR= f"{GCS_BUCKET}/{MODEL_NAME}-{TIMESTAMP}"

print(DISPLAY_NAME)
print(GCS_BASE_OUTPUT_DIR)

bert-sentiment-20211008165901
gs://dougkelly-vertex-demos-vertex-challenge-lab/bert-sentiment-classifier-20211008165901


In [222]:
SERVING_IMAGE_URI = 'us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-5:latest'

In [217]:
# command line args for trainer.task defined above. Review the 'help' argument for a description.
# You will set the model training args below. Vertex AI will set the environment variables for training URIs.
# CMD_ARGS= [
#     "--learning-rate=" + str(0.001),
#     "--batch-size=" + str(16),
#     "--n-train-examples=" + str(2638),
#     "--stop-point=" + str(10),
#     "--n-checkpoints=" + str(10),
#     "--dropout=" + str(0.2),   
# ]

In [ ]:
job = vertexai.CustomContainerTrainingJob(
    display_name=MODEL_NAME,
    container_uri=IMAGE_URI,
    model_serving_container_image_uri=SERVING_IMAGE_URI,
    # https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers
    # model_serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-3:latest",
)

model = job.run(
    # dataset=tabular_dataset,
    # model_display_name=DISPLAY_NAME,
    # GCS custom job output dir.
    base_output_dir=GCS_BASE_OUTPUT_DIR,
    # args=CMD_ARGS,
    # Custom job WorkerPool arguments.
    replica_count=1,
    machine_type="c2-standard-4",
    # Provide your Tensorboard resource name to write Tensorboard logs during training.
    # tensorboard=TENSORBOARD_RESOURCE_NAME,
    # Provide your Vertex custom training service account created during lab setup.
    # service_account=f"vertex-custom-training-sa@{PROJECT_ID}.iam.gserviceaccount.com"
)

## Define a pipeline using the KFP SDK

In [228]:
@kfp.dsl.pipeline(name="bert-sentiment-classification", pipeline_root=PIPELINE_ROOT)
def pipeline(
    project: str = PROJECT_ID,
    gcp_region: str = REGION,
    model_container_uri: str = IMAGE_URI,
    model_serving_container_uri: str = SERVING_IMAGE_URI,
    # cmd_args: list = CMD_ARGS,
    staging_bucket: str = GCS_BUCKET,
    gcs_base_output_dir: str = GCS_BASE_OUTPUT_DIR,
    model_display_name: str = DISPLAY_NAME,
):
    
    model_train_evaluate_op = gcc_aip.CustomContainerTrainingJobRunOp(
        display_name=model_display_name,
        container_uri=model_container_uri,
        model_serving_container_image_uri=model_serving_container_uri,
        # args=cmd_args,
        staging_bucket=staging_bucket,
        base_output_dir=gcs_base_output_dir,
        # Custom job WorkerPool arguments.
        replica_count=1,
        machine_type="c2-standard-4",        
    )
    
    model_deploy_op = gcc_aip.ModelDeployOp(
        model=model_train_evaluate_op.outputs["model"],
        project=project,
        location=gcp_region,
        machine_type="n1-standard-4",
    )

## Compile the pipeline using the KFP SDK

In [229]:
from kfp.v2 import compiler

In [230]:
compiler.Compiler().compile(
    pipeline_func=pipeline, package_path="bert-sentiment-classification.json"
)

## Run the pipeline on Vertex Pipelines

In [231]:
vertex_pipelines_job = vertexai.pipeline_jobs.PipelineJob(
    display_name="bert-sentiment-classification",
    template_path="bert-sentiment-classification.json",
    parameter_values={"project": PROJECT_ID,
                      "gcp_region": REGION,
                      "model_container_uri": IMAGE_URI,
                      "model_serving_container_uri": SERVING_IMAGE_URI,
                      "staging_bucket": GCS_BUCKET, 
                      "gcs_base_output_dir": GCS_BASE_OUTPUT_DIR,
                      "model_display_name": DISPLAY_NAME},
    enable_caching=True,
)

In [232]:
vertex_pipelines_job.run()

INFO:google.cloud.aiplatform.pipeline_jobs:Creating PipelineJob
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob created. Resource name: projects/617979904441/locations/us-central1/pipelineJobs/bert-sentiment-classification-20211008170707
INFO:google.cloud.aiplatform.pipeline_jobs:To use this PipelineJob in another session:
INFO:google.cloud.aiplatform.pipeline_jobs:pipeline_job = aiplatform.PipelineJob.get('projects/617979904441/locations/us-central1/pipelineJobs/bert-sentiment-classification-20211008170707')
INFO:google.cloud.aiplatform.pipeline_jobs:View Pipeline Job:
https://console.cloud.google.com/vertex-ai/locations/us-central1/pipelines/runs/bert-sentiment-classification-20211008170707?project=617979904441
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projects/617979904441/locations/us-central1/pipelineJobs/bert-sentiment-classification-20211008170707 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.pipeline_jobs:PipelineJob projec

RuntimeError: Job failed with:
code: 9
message: "The DAG failed because some tasks failed. The failed tasks are: [customcontainertrainingjob-run].; Job (project_id = dougkelly-vertex-demos, job_id = 3339040891695267840) is failed due to the above error.; Failed to handle the job: {project_number = 617979904441, job_id = 3339040891695267840}"


## Query deployed model on Vertex Endpoint for online predictions